<a href="https://colab.research.google.com/github/Jathu03/Assignment-02/blob/main/Assignement_2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

def pairwise_squared_euclidean_distance(X, Y):
    # ||x_i - y_j||^2 = ||x_i||^2 + ||y_j||^2 - 2 * <x_i,y_j>
    # Computing the squared norms of matrices X and Y
    X_norm_squared = np.sum(X**2, axis=1).reshape(-1, 1)  # getting norm of X as a column vector of shape =(m, 1)
    Y_norm_squared = np.sum(Y**2, axis=1).reshape(1, -1)  # getting norm of Y as a row vector of shape = (1, n)

    # Computing the dot product of X and Y
    X_Y_dot_product = np.dot(X, Y.T)

    return X_norm_squared + Y_norm_squared - 2 * X_Y_dot_product


X = np.random.randn(5, 3)  # 5 x 3 matrix
Y = np.random.randn(6, 3)  # 6 x 3 matrix

Z = pairwise_squared_euclidean_distance(X, Y)

print(Z)


[[ 1.02069749  6.77852076  4.15663733  6.43700597  4.88184863  6.62187736]
 [ 9.96512272  8.59341755  2.20121959 14.94321186  4.39381488 12.1101634 ]
 [ 0.2578211   7.62028864  8.95066564  3.90943138  7.23967365  4.00427178]
 [ 7.25978437  0.30973006  9.17800063  3.16384683  0.12724513  3.35101603]
 [ 5.39087401  3.39382984  3.45189363  6.54164954  1.1676055   3.0312488 ]]


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
import numpy as np

# New Section

In [ ]:
# Code to load the dataset and extract feature embedding

# Define the transformation to be applied to the images
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset from Google Drive
dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform)

# Split dataset randomly into training and testing
train_size = int(2/3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load Pre-trained ResNet-50 Model
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove the last classification layer
resnet50.eval()  # Set model to evaluation mode

# Extract the feature embeddings
def extract_features(data_loader, model):
    features = []
    labels = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten the output tensor
            features.append(outputs.numpy())
            labels.append(targets.numpy())

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

# Extract features from train and test datasets
train_features, train_labels = extract_features(train_loader, resnet50)
test_features, test_labels = extract_features(test_loader, resnet50)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can als

In [23]:
#  k-NN Classification (Question 3.a)
def knn_predict(test_feature, train_features, train_labels, k):

  distances = pairwise_squared_euclidean_distance(train_features, test_feature)  # Calculate Euclidean distances
  k_nearest_indices = np.argsort(distances)[:k]  # Find k nearest neighbors

  # Majority vote for the class label
  class_counts = np.bincount(train_labels[k_nearest_indices])
  return np.argmax(class_counts)


# Evaluate on Test Set
test_predictions = knn.predict(test_features)
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Test Set Accuracy: {accuracy * 100:.2f}%")

Test Set Accuracy: 90.72%


In [24]:
# Train Linear Classifier (Question 3.b)

# Convert features and labels into tensors
train_features_tensor = torch.tensor(train_features, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)

# Define the linear classifier
num_classes = len(dataset.classes)
input_dim = train_features.shape[1]

classifier = nn.Linear(input_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Training for the linear classifier
epochs = 20
batch_size = 32
num_samples = train_features_tensor.shape[0]

for epoch in range(epochs):
    classifier.train()
    total_loss = 0

    # Mini-batch training
    permutation = torch.randperm(num_samples)
    for i in range(0, num_samples, batch_size):
        indices = permutation[i:i + batch_size]
        batch_features = train_features_tensor[indices]
        batch_labels = train_labels_tensor[indices]

        # Forward pass
        outputs = classifier(batch_features)
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Evaluate the Test Set
classifier.eval()
test_features_tensor = torch.tensor(test_features, dtype=torch.float32).to(device)

with torch.no_grad():
    test_outputs = classifier(test_features_tensor)
    _, test_predictions = torch.max(test_outputs, 1)

test_accuracy = accuracy_score(test_labels, test_predictions.cpu().numpy())
print(f"Test Set Accuracy: {test_accuracy * 100:.2f}%")

Epoch [1/20], Loss: 248.7729
Epoch [2/20], Loss: 51.8058
Epoch [3/20], Loss: 31.4364
Epoch [4/20], Loss: 19.5602
Epoch [5/20], Loss: 14.1242
Epoch [6/20], Loss: 10.2553
Epoch [7/20], Loss: 8.0954
Epoch [8/20], Loss: 5.8857
Epoch [9/20], Loss: 5.0249
Epoch [10/20], Loss: 4.3128
Epoch [11/20], Loss: 3.3955
Epoch [12/20], Loss: 3.2046
Epoch [13/20], Loss: 2.8537
Epoch [14/20], Loss: 2.2745
Epoch [15/20], Loss: 2.3045
Epoch [16/20], Loss: 1.8016
Epoch [17/20], Loss: 1.7371
Epoch [18/20], Loss: 1.5970
Epoch [19/20], Loss: 1.4978
Epoch [20/20], Loss: 1.2606
Test Set Accuracy: 92.62%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Step 1: Load and Split the Caltech-101 Dataset with Data Augmentation
# Data augmentation for better generalization
data_transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the entire dataset and split it into training and testing
dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform_train)
train_size = int(2 / 3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Apply test transform to the test dataset
test_dataset.dataset.transform = data_transform_test

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Step 2: Load Pre-trained ResNet-50 Model and Modify for Fine-tuning
resnet50 = models.resnet50(pretrained=True)

# Modify the fully connected layer to match the number of classes in Caltech-101
num_classes = len(dataset.classes)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)

# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.0001)  # Use a small learning rate for fine-tuning
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # Learning rate scheduler

# Step 4: Train the Network
epochs = 10
for epoch in range(epochs):
    resnet50.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()  # Adjust the learning rate
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Step 5: Evaluate on Test Set
resnet50.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet50(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Set Accuracy: {test_accuracy:.2f}%")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 299.9618, Train Accuracy: 68.64%
Epoch [2/10], Loss: 46.1127, Train Accuracy: 95.60%
Epoch [3/10], Loss: 13.3897, Train Accuracy: 98.70%
Epoch [4/10], Loss: 9.1128, Train Accuracy: 99.13%
Epoch [5/10], Loss: 6.2251, Train Accuracy: 99.48%
Epoch [6/10], Loss: 2.8371, Train Accuracy: 99.75%
